In [1]:
!git clone https://github.com/jasonlaska/spherecluster

Cloning into 'spherecluster'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 686 (delta 4), reused 6 (delta 2), pack-reused 672
Receiving objects: 100% (686/686), 1.64 MiB | 10.87 MiB/s, done.
Resolving deltas: 100% (434/434), done.


In [2]:
cd spherecluster

/content/spherecluster


In [3]:
!python setup.py install

running install
running bdist_egg
running egg_info
creating spherecluster.egg-info
writing spherecluster.egg-info/PKG-INFO
writing dependency_links to spherecluster.egg-info/dependency_links.txt
writing requirements to spherecluster.egg-info/requires.txt
writing top-level names to spherecluster.egg-info/top_level.txt
writing manifest file 'spherecluster.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
writing manifest file 'spherecluster.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/spherecluster
copying spherecluster/spherical_kmeans.py -> build/lib/spherecluster
copying spherecluster/__init__.py -> build/lib/spherecluster
copying spherecluster/von_mises_fisher_mixture.py -> build/lib/spherecluster
copying spherecluster/util.py -> build/lib/spherecluster
creating build/bdist.linux-x86_64
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-

In [4]:
!pip install spherecluster

In [5]:
#참고: https://yamalab.tistory.com/118

from keras.datasets import mnist
import numpy as np

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))
print (x_train.shape)
print (x_test.shape)

Using TensorFlow backend.


11493376/11490434 [==============================] - 0s 0us/step
(60000, 784)
(10000, 784)


In [6]:
from keras.layers import Input, Dense
from keras.models import Model
from keras import backend as K

# configure
encoding_dim = 200
input_img = Input(shape=(784,))

# layers
encoded = Dense(encoding_dim, activation='relu')(input_img)
decoded = Dense(784, activation='sigmoid')(encoded)


# Models
autoencoder = Model(input_img, decoded) # autoencoder

encoder = Model(input_img, encoded) # encoder

encoded_input = Input(shape=(encoding_dim,))
decoder_layer = autoencoder.layers[-1]
decoder = Model(encoded_input, decoder_layer(encoded_input)) # decoder

In [7]:
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1))

def recall(y_true, y_pred):
    y_true_yn = K.round(K.clip(y_true, 0, 1))
    y_pred_yn = K.round(K.clip(y_pred, 0, 1))

    count_true_positive = K.sum(y_true_yn * y_pred_yn)
    count_true_positive_false_negative = K.sum(y_true_yn)
    recall = count_true_positive / (count_true_positive_false_negative + K.epsilon())
    return recall


# train autoencoder
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy', metrics=[rmse, recall])
autoencoder.fit(x_train, x_train,
                epochs=100,
                batch_size=512,
                shuffle=True,
                validation_data=(x_test, x_test))


# encoding result
encoded_imgs = encoder.predict(x_train)
decoded_imgs = decoder.predict(encoded_imgs)

encoded_imgs.shape



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 60000 samples, validate on 10000 samples
Epoch 1/100





60000/60000 [==============================] - 10s 173us/step - loss: 0.3968 - rmse: 0.3209 - recall: 0.2727 - val_loss: 0.2744 - val_rmse: 0.2596 - val_recall: 0.2219
Epoch 2/100
60000/60000 [==============================] - 1s 13us/step - loss: 0.2668 - rmse: 0.2550 - recall: 0.2413 - val_loss: 0.2575 - val_rmse: 0.2494 - val_recall: 0.2638
Epoch 3/100
60000/60000 [==============================] - 1s 13us/step - loss: 0.2500 - rmse: 0.2433 - recall: 0.3324 - val_loss: 0.2398 - val_rmse: 0.2358 - val_recall: 0.4065
Epoch 4/100
60000/60000 [==============================] - 1s 12us/step - loss: 0.2329 - rmse: 0.2297 - recall: 0.4467 - val_loss: 0.2233 - val_rmse: 0.2226 - val_recall: 0.5178
Epoch 5/100
60000/60000 [==============================] - 1s 12us/step - loss: 0.2182 - rmse: 0.2180 - recall: 0.5287 - val_loss: 0

(60000, 200)

In [0]:
from spherecluster import SphericalKMeans

skm = SphericalKMeans(n_clusters=10)
skm.fit(x_train)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.cluster.k_means_ module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.cluster. Anything that cannot be imported from sklearn.cluster is now part of the private API.
  warnings.warn(message, FutureWarning)


In [0]:
import matplotlib.pyplot as plt

def viz_img(y_pred):
    n = 10
    fig = plt.figure(1)
    box_index = 1
    for cluster in range(10):
        result = np.where(y_pred == cluster)
        for i in np.random.choice(result[0].tolist(), n, replace=False):
            ax = fig.add_subplot(n, n, box_index)
            plt.imshow(x_train[i].reshape(28, 28))
            plt.gray()
            ax.get_xaxis().set_visible(False)
            ax.get_yaxis().set_visible(False)
            box_index += 1
    plt.show()
    
viz_img(skm.labels_)